# Учимся распознавать рукописные цифры с помощью нейронной сети

Теперь мы достигли точки, когда мы можем решить очень интересную задачу: применить полученные знания в области машинного обучения в целом и `Flux.jl` в частности для создания нейронной сети, способной распознавать рукописные цифры! Данные взяты из набора данных под названием MNIST, который стал классикой в мире машинного обучения. 

*Вместо этого мы могли бы попробовать применить методы к оригинальным изображениям фруктов. Однако изображения фруктов намного больше изображений MNIST, что делает обучение подходящей нейронной сети слишком медленным.*

[Можно поиграться](https://fluxml.ai/experiments/mnist/)

## Обработка данных

Как мы знаем, первая трудность с любым новым набором данных - это найти его, понять, в каком формате он хранится, прочитать его и декодировать в полезную структуру данных в Julia. 

Исходные данные MNIST доступны [здесь](http://yann.lecun.com/exdb/mnist); см. также [страница Википедии](https://ru.wikipedia.org/wiki/MNIST_(база_данных)). Однако формат, в котором хранятся данные, довольно неясен. 

К счастью, различные пакеты в Julia предоставляют более удобный интерфейс для доступа к нему. 

Мы будем использовать тот, который предоставлен Flux.jl. Данные представляют собой изображения рукописных цифр и соответствующие метки, которые были определены вручную (то есть людьми). Наша работа состоит в том, чтобы заставить компьютер **учиться** распознавать цифры, изучая, как обычно, функцию, которая связывает входные и выходные данные.

### Загрузка и проверка данных

Сначала мы загружаем необходимые пакеты:

In [1]:
using Flux, Flux.Data.MNIST

Загружаем данные:

In [ ]:
labels = MNIST.labels();
images = MNIST.images();  # точка с запятой (`;`) здесь важно: это мешает Юлии показывать объект

#### Упражнение 1

Изучите данные «меток». Затем изучите первые несколько изображений. *Не пытайтесь просмотреть весь объект `images`!* Попробуйте углубиться в подробности, чтобы выяснить, как располагаются данные.

In [ ]:
images[8]

In [ ]:
using Images

In [ ]:
channels = Float64.(channelview(images[8]))
Gray.(channels)

#### Упражнение 2

Convert the first image to a matrix of `Float64`.

### Анализ данных

В предыдущих записных книжках мы организовали входные данные для Flux как массив массивов. Теперь мы будем использовать альтернативное представление в виде матрицы, поскольку это позволяет `Flux` использовать матричные операции, которые более эффективны. 

Столбец $ i $ матрицы - это вектор, состоящий из $ i $-ой точки данных $ \mathbf {x} ^ {(i)} $. Точно так же желаемые выходные данные представлены в виде матрицы, причем $ i $-й столбец является желаемым выходным значением $ \mathbf {y} ^ {(i)} $.

#### Упражнение 3

Изображение - это матрица цветов, но теперь нам нужен вектор. Для этого мы просто упорядочиваем все элементы матрицы определенным образом в единый список; к счастью, Юлия уже предоставляет для этого функцию `vec`!

1. Какой порядок использует `vec`? *Это отражает основной способ хранения матрицы в памяти.*

2. Как вы можете конвертировать изображение в вектор `Float64`?

3. Определите переменную $ n $, которая является длиной этих векторов.

#### Упражнение 4

Создайте функцию `rewrite`, которая принимает диапазон и преобразует этот диапазон изображений в векторы с плавающей точкой и размещает их горизонтально, используя` hcat` и оператор 'splat' `...`.

Нам также нужна матрица из горячих векторов. `Flux` предоставляет функцию` onehotbatch` для этого (вам нужно будет ее импортировать). Он работает как `onehot`, но принимает вектор меток и выводит матрицу` Y`. 

Вернуть пару `(X, Y)`.

## Настройка нейронной сети

Теперь мы должны настроить нейронную сеть. Поскольку данные сложны, мы можем ожидать, что потребуется несколько слоев. Но мы можем начать с одного слоя. 

- Сеть будет принимать в качестве входных данных векторы $ \mathbf {x} ^ {(i)} $, поэтому входной слой имеет $ n $ узлов. 

- на выходе будет горячий вектор, кодирующий требуемую цифру от 1 до 9 или 0. Существует 10 возможных категорий, поэтому нам нужен выходной слой размером 10. 

Тогда наша задача как разработчиков нейронных сетей - вставить слои между этими входными и выходными слоями, вес которых будет настроен в процессе обучения. *Это искусство, а не наука*! Но в основном успех зависит от арчитектуры сети.

### Softmax

Мы сделаем сеть с одним слоем; давайте выберем каждый нейрон в слое, чтобы использовать функцию активации `relu`. Выходной сигнал `relu` может быть сколь угодно большим, но в конце мы захотим сравнить выходной сигнал сети с горячими векторами, то есть значениями между $ 0 $ и $ 1 $. 

Чтобы выполнить эту работу, мы будем использовать дополнительную функцию в конце, которая берет вектор произвольных действительных чисел и отображает его («раздавливает») в вектор чисел между $ 0 $ и $ 1 $. 

Наиболее часто используемая функция с этим свойством - $ \mathrm {softmax} $. Сначала мы берем экспоненту каждой входной переменной, чтобы сделать их положительными. Затем мы делим на сумму, чтобы убедиться, что они лежат между $ 0 $ и $ 1 $.

$$\mathrm{softmax}(\mathbf{x})_i := \frac{\exp (x_i)}{\sum_j \exp(x_j)}$$

Обратите внимание, что здесь мы записали результат для $ i $ -ого компонента функции $ \mathbf {R} ^ n \to \mathbf {R} ^ n $. Также обратите внимание, что функция возвращает вектор чисел, которые являются положительными, и чьи компоненты составляют сумму $ 1 $. 

Таким образом, фактически их можно рассматривать как вероятности. В контексте нейронной сети использование `softmax` после последнего слоя, таким образом, позволяет интерпретировать выходные данные как вероятности, в нашем случае вероятность того, что сеть назначит, что данное изображение представляет каждое возможное выходное значение ($ 0 $ - $ 9 $)!

#### Упражнение 5

Создайте нейронную сеть с одним слоем, используя функцию $ \sigma $ и выходом `softmax`.

## Обучение

Как мы знаем, **обучение** состоит из итеративной настройки параметров модели для уменьшения функции «потерь». Какие параметры нужно отрегулировать? Все! 

Поскольку функция `loss` содержит вызов функции` model`, вызов `back!` на результате функции потерь обновляет информацию о градиенте функции потерь относительно *каждого узла в сети!*:

In [ ]:
l = loss(X, Y)

Flux.Tracker.back!(l)

Это то, что происходит внутри функции `train!`. На самом деле `train! (Loss, data, opt)` выполняет итерацию по каждому объекту в `data` и запускает эту функцию. По этой причине `data` должен состоять из повторяемого объекта, который возвращает пары` (X, Y) `на каждом шаге.

Самая простая возможность

In [ ]:
data = ((X, Y), )  # one-element tuple

В качестве альтернативы, мы можем сделать один вызов функции `train!`, Повторить несколько копий `data`, используя` repeat`. Это **итератор**; он не копирует данные 100 раз, что было бы очень расточительно; он просто дает объект, который многократно повторяет одни и те же данные:

In [ ]:
dataset = Base.Iterators.repeated((X, Y), 100)

#### Упражнение 6

Обучите модель на подмножестве $ N $ изображений при $N = 5000$.

In [ ]:
N = 5_000
X, Y = rewrite(1:N)

Функция `loss`, вычисленная на матрицах, дает общую погрешность:

In [ ]:
loss(X, Y)

In [ ]:
@time Flux.train!(loss, data, opt)

In [ ]:
@time Flux.train!(loss, dataset, opt)

Это (приблизительно) эквивалентно простому выполнению цикла `for` для запуска предыдущей команды` train! `100 раз.

### Использование обратных вызовов

Функция `train!` может принимать необязательный аргумент ключевое слово `cb` (сокращение от `callback`). Функция обратного вызова - это функция, которую вы предоставляете в качестве аргумента функции `f`, которая очень часто вызывает вашу функцию. 

Это дает возможность предоставлять функцию, которая вызывается на каждом этапе или время от времени в процессе обучения. Распространенным вариантом использования является визуальное отслеживание процесса обучения путем распечатки текущего значения функции `loss`:

In [ ]:
callback() = @show(loss(X, Y))

Flux.train!(loss, data, opt; cb = callback)

In [ ]:
Flux.train!(loss, dataset, opt; cb = callback)

Однако вычисление полной функции «потерь» обходится дорого, и нет необходимости выводить ее каждый шаг. Так что `Flux` также предоставляет функцию` throttle`, которая предоставляет механизм для вызова данной функции не чаще, чем раз в определенное количество шагов:

In [ ]:
Flux.train!(loss, dataset, opt; cb = Flux.throttle(callback, 1))

In [ ]:
for i in 1:100
    Flux.train!(loss, dataset, opt; cb = Flux.throttle(callback, 1))
end

## Этап тестирования

Теперь мы обучили модель, то есть мы нашли параметры `W` и` b` для сетевого уровня (уровней). Чтобы **проверить**, была ли процедура обучения действительно успешной, мы проверяем, насколько хорошо работает получившаяся обученная сеть, когда мы тестируем ее на изображениях, которые сеть еще не видела! 

Часто набор данных для этой цели разделяется на «данные обучения» и «данные тестирования (или проверки)», и, действительно, набор данных MNIST имеет отдельный пул данных обучения. Вместо этого мы можем использовать изображения, которые мы не включили в сокращенный учебный процесс.

#### Упражнение 7

Возьмите следующие 100 изображений после тех, которые были использованы для обучения. Насколько хорошо они предсказываются?

In [ ]:
X_test, Y_test = rewrite(N+1:N+100)

In [ ]:
loss(X_test, Y_test)

In [ ]:
display(images[N+1])
labels[N+1]

In [ ]:
[model(X_test[:,1]) Y_test[:,1]]

In [ ]:
loss(X_test[:,1], Y_test[:,1])

In [ ]:
loss(X_test, Y_test)

#### Упражнение 8

Используйте функцию «indmax», чтобы написать функцию `prediction`, которая сообщает, какую цифра предсказывает «модель», как индекс с максимальным весом.

#### Упражнение 9

Подсчитайте количество правильных прогнозов по всему набору данных и, следовательно, процент изображений, которые были правильно предсказаны. *Этот процент используется для сравнения различных методов машинного обучения.*

## Улучшение прогноза

До сих пор мы использовали один слой. Чтобы улучшить прогнозирование, нам, вероятно, нужно использовать больше слоев.

#### Упражнение 10

Введите промежуточный скрытый слой. Дает ли это лучший прогноз?